<a href="https://colab.research.google.com/github/raghurammanikanta/WORK-ON-TELUGU-DATA/blob/main/complete_work_on_telugu_news_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install indic-nlp-library

# Import Libraries and Load Dataset

In [ ]:
import pandas as pd
import re
import numpy as np
from gensim.models import FastText, Word2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.ensemble import RandomForestClassifier
from indicnlp.tokenize import indic_tokenize



In [ ]:
# Load and preprocess dataset
df = pd.read_csv('/content/TELUGU_NEWS_TEST.csv')
df.head()
df['topic'] = df['topic'].apply(lambda x: x if x in ['entertainment', 'nation'] else 'others')
c = df['topic'].value_counts()
c


topic
nation           1673
others           1367
entertainment    1289
Name: count, dtype: int64

# **Text Preprocessing**

In [ ]:
def pre_pro(text):
    text = re.sub(r'[^ఁ-౿ ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['body'] = df['body'].apply(pre_pro)
print("\nPreprocessed Text:")
print(df.head())


Preprocessed Text:
     SNo                 date  \
0   7771  19-05-2017 13:44:10   
1   9591  01-08-2017 13:58:24   
2  12622  05-04-2017 00:31:58   
3  16401  30-08-2017 03:34:37   
4  10422  10-04-2017 09:00:48   

                                             heading  \
0                                               కేశవ   
1  డబ్బుల కోసం ప్రాణాల మీదకు తెచ్చుకుంటారా.. హీరో...   
2                    దక్షిణాదిన బీజేపీ పప్పులుడకవ్‌    
3                 పీఎస్‌ఎల్వీ-సీ39కి గ్రీన్‌సిగ్నల్‌   
4  ఏయ్ జయప్రకాష్ అనే కేక వినిపించింది.. వెనక్కి త...   

                                                body          topic  
0  హీరోగా తెలుగు సినిమాల్లోకి ఎంట్రీ ఇచ్చిన నిఖిల...  entertainment  
1  సినిమాల్లో యాక్షన్ స్టంట్లు చేసేటప్పుడు ఎక్కువ...  entertainment  
2  దక్షిణ భారతాన్ని ఆక్రమించేందుకు బీజేపీ పంచెలు ...         nation  
3  నేటి మధ్యాహ్నం కి కౌంట్డౌన్ షురూశ్రీహరికోట ఆగస...         nation  
4  ఏమి రా అబ్బి యాడికి పోయినావు అంటూ రాయలసీమ యాసన...  entertainment  


FastText Model Training and Embedding Extraction

In [ ]:
df['body'] = df['body'].apply(lambda x: x.split())
embedding_size = 100 ##embedding_size is the size of the embedding vector.
window_size = 5 #window_size is the size of the number of words
min_word = 6 #min_word, which specifies the minimum frequency of a word
down_sampling = 1e-2 #most frequently occurring word will be down-sampled by a number specified by the down_sampling attribute


ft_model = FastText(sentences = df['body'],vector_size=embedding_size,window=window_size,min_count=min_word,sample=down_sampling,sg=1,epochs=10)
#sg defines if the parameter is skpgram or cbow

print(ft_model.wv)

# Extract word embeddings
X = []
for sentence in df['body']:
    word_embeddings = [ft_model.wv[word] for word in sentence if word in ft_model.wv]
    if word_embeddings:
        sentence_embedding = np.mean(word_embeddings, axis=0)
        X.append(sentence_embedding)
    else:
        X.append(np.zeros(embedding_size))

X = np.array(X)


FastTextKeyedVectors<vector_size=100, 16282 keys>


TF-IDF Embedding Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the tokenized words back into sentences
df['text_joined'] = df['body'].apply(lambda x: ' '.join(x))

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_joined'])

# `tfidf_matrix` now contains your TF-IDF embeddings
print(tfidf_matrix)

  (0, 5168)	0.06070476226620654
  (0, 3607)	0.05400373939202602
  (0, 2224)	0.03709842715448317
  (0, 1030)	0.05511371558487567
  (0, 746)	0.04965128960888237
  (0, 4717)	0.03340671972725232
  (0, 2535)	0.025061712619676796
  (0, 4646)	0.05436043153595191
  (0, 1176)	0.05253748783705576
  (0, 2248)	0.05400373939202602
  (0, 1225)	0.07891742938490594
  (0, 3217)	0.0684516743401461
  (0, 3209)	0.03491484879890116
  (0, 1099)	0.03978189929167606
  (0, 1742)	0.034685125687291596
  (0, 2651)	0.021518894057083474
  (0, 3584)	0.03490522998343258
  (0, 4403)	0.021001460102973747
  (0, 4450)	0.04126980087199144
  (0, 997)	0.06534756666109966
  (0, 4727)	0.061302808200556266
  (0, 461)	0.028177443722581918
  (0, 4339)	0.028643730534997587
  (0, 1202)	0.02149739665804461
  (0, 2520)	0.026223425569685106
  :	:
  (4328, 3720)	0.10193484367823911
  (4328, 800)	0.07924725908723386
  (4328, 3953)	0.0595326739373003
  (4328, 1280)	0.027572565385999184
  (4328, 392)	0.05724895167842428
  (4328, 5629)	0.

In [ ]:
TY = tfidf_matrix

Word2Vec Model Training and Embedding Extraction

In [ ]:

from indicnlp.tokenize import indic_tokenize

# Assuming df['body'] contains your Telugu text data
df = pd.read_csv('/content/TELUGU_NEWS_TEST.csv')  # Load your Telugu data

# Tokenize Telugu sentences
sentences = [indic_tokenize.trivial_tokenize(sentence.lower()) for sentence in df['body']]

# CBOW model
cbow_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Skip-gram model
sg_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Save the models
cbow_model.save("telugu_word2vec_cbow.model")
sg_model.save("telugu_word2vec_sg.model")
w2v_model = Word2Vec.load("telugu_word2vec_cbow.model")

# Function to get document vector
def get_document_vector(doc):
    words = doc.split()
    word_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(word_vecs) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(word_vecs, axis=0)

# Create document vectors
X_w2v = np.array([get_document_vector(doc) for doc in df['body']])
yw = df['topic']

 Train-Test Split

In [ ]:
# Split the data
WX_train, WX_test, Wy_train, Wy_test = train_test_split(X_w2v, yw, test_size=0.2, random_state=42)
TX_train, TX_test, Ty_train, Ty_test = train_test_split(TY, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


 SVM Model Training and Evaluation

In [ ]:
# SVM with TF-IDF
clf = SVC(kernel='linear')  # You can explore other kernels as well
clf.fit(TX_train, Ty_train)
Ty_pred = clf.predict(TX_test)
print(f'\nSVM with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# SVM with FastText
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'\nSVM with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# SVM with Word2Vec
svm_model = SVC(kernel='linear')
svm_model.fit(WX_train, Wy_train)
yw_pred = svm_model.predict(WX_test)
print(f'\nSVM with Word2Vec Accuracy: {accuracy_score(Wy_test, y_pred)}')
print(classification_report(Wy_test, yw_pred))



SVM with TF-IDF Accuracy: 0.8602771362586605
               precision    recall  f1-score   support

entertainment       0.94      0.90      0.92       237
       nation       0.85      0.86      0.85       357
       others       0.81      0.83      0.82       272

     accuracy                           0.86       866
    macro avg       0.87      0.86      0.86       866
 weighted avg       0.86      0.86      0.86       866


SVM with FastText Accuracy: 0.9122401847575058
               precision    recall  f1-score   support

entertainment       0.98      0.96      0.97       237
       nation       0.92      0.87      0.90       357
       others       0.85      0.92      0.88       272

     accuracy                           0.91       866
    macro avg       0.92      0.92      0.92       866
 weighted avg       0.91      0.91      0.91       866


SVM with Word2Vec Accuracy: 0.6235565819861432
               precision    recall  f1-score   support

     business       0.70  

KNN Model Training and Evaluation

In [ ]:
# KNN with FastText
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)
print(f'\nKNN with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# KNN with TF-IDF
knn_classifier.fit(TX_train, Ty_train)
Ty_pred = knn_classifier.predict(TX_test)
print(f'\nKNN with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# KNN with Word2Vec
knn_classifier.fit(WX_train, Wy_train)
y_pred = knn_classifier.predict(WX_test)
print(f'\nKNN with Word2Vec Accuracy: {accuracy_score(Wy_test, y_pred)}')
print(classification_report(Wy_test, y_pred))



KNN with FastText Accuracy: 0.9260969976905312
               precision    recall  f1-score   support

entertainment       0.99      0.96      0.98       237
       nation       0.93      0.90      0.91       357
       others       0.87      0.93      0.90       272

     accuracy                           0.93       866
    macro avg       0.93      0.93      0.93       866
 weighted avg       0.93      0.93      0.93       866


KNN with TF-IDF Accuracy: 0.3556581986143187
               precision    recall  f1-score   support

entertainment       0.95      0.08      0.15       237
       nation       0.90      0.05      0.10       357
       others       0.33      1.00      0.49       272

     accuracy                           0.36       866
    macro avg       0.73      0.38      0.25       866
 weighted avg       0.73      0.36      0.23       866


KNN with Word2Vec Accuracy: 0.7575057736720554
               precision    recall  f1-score   support

     business       0.62  

Logistic Regression Model Training and Evaluation

In [ ]:
# Logistic Regression with FastText
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)
y_pred = logreg_classifier.predict(X_test)
print(f'\nLogistic Regression with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Logistic Regression with TF-IDF
logreg_classifier.fit(TX_train, Ty_train)
Ty_pred = logreg_classifier.predict(TX_test)
print(f'\nLogistic Regression with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Logistic Regression with Word2Vec
logreg_classifier.fit(WX_train, Wy_train)
Wy_pred = logreg_classifier.predict(WX_test)
print(f'\nLogistic Regression with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Logistic Regression with FastText Accuracy: 0.9064665127020786
               precision    recall  f1-score   support

entertainment       0.98      0.95      0.97       237
       nation       0.91      0.88      0.89       357
       others       0.84      0.90      0.87       272

     accuracy                           0.91       866
    macro avg       0.91      0.91      0.91       866
 weighted avg       0.91      0.91      0.91       866


Logistic Regression with TF-IDF Accuracy: 0.8614318706697459
               precision    recall  f1-score   support

entertainment       0.94      0.92      0.93       237
       nation       0.85      0.86      0.86       357
       others       0.81      0.81      0.81       272

     accuracy                           0.86       866
    macro avg       0.87      0.86      0.87       866
 weighted avg       0.86      0.86      0.86       866


Logistic Regression with Word2Vec Accuracy: 0.766743648960739
               precision    recall 

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Decision Tree Model Training and Evaluation

In [ ]:
# Decision Tree with FastText
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
y_pred = dt_classifier.predict(X_test)
print(f'\nDecision Tree with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Decision Tree with TF-IDF
dt_classifier.fit(TX_train, Ty_train)
Ty_pred = dt_classifier.predict(TX_test)
print(f'\nDecision Tree with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Decision Tree with Word2Vec
dt_classifier.fit(WX_train, Wy_train)
Wy_pred = dt_classifier.predict(WX_test)
print(f'\nDecision Tree with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Decision Tree with FastText Accuracy: 0.8314087759815243
               precision    recall  f1-score   support

entertainment       0.92      0.89      0.91       237
       nation       0.82      0.80      0.81       357
       others       0.77      0.82      0.79       272

     accuracy                           0.83       866
    macro avg       0.84      0.84      0.84       866
 weighted avg       0.83      0.83      0.83       866


Decision Tree with TF-IDF Accuracy: 0.6212471131639723
               precision    recall  f1-score   support

entertainment       0.69      0.73      0.71       237
       nation       0.63      0.58      0.60       357
       others       0.55      0.58      0.57       272

     accuracy                           0.62       866
    macro avg       0.62      0.63      0.63       866
 weighted avg       0.62      0.62      0.62       866


Decision Tree with Word2Vec Accuracy: 0.7032332563510393
               precision    recall  f1-score   suppo

Naive Bayes Model Training and Evaluation

In [ ]:
# Naive Bayes with FastText
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_scaled, y_train)
y_pred = nb_classifier.predict(X_test_scaled)
print(f'\nNaive Bayes with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Naive Bayes with TF-IDF
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(TX_train)
X_test_scaled = scaler.transform(TX_test)
nb_classifier.fit(X_train_scaled, Ty_train)
Ty_pred = nb_classifier.predict(X_test_scaled)
print(f'\nNaive Bayes with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Naive Bayes with Word2Vec
scaler = MinMaxScaler()
WX_train_scaled = scaler.fit_transform(WX_train)
WX_test_scaled = scaler.transform(WX_test)
nb_classifier.fit(WX_train_scaled, Wy_train)
Wy_pred = nb_classifier.predict(WX_test_scaled)
print(f'\nNaive Bayes with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Naive Bayes with FastText Accuracy: 0.8452655889145496
               precision    recall  f1-score   support

entertainment       0.98      0.90      0.94       237
       nation       0.76      0.95      0.84       357
       others       0.90      0.66      0.76       272

     accuracy                           0.85       866
    macro avg       0.88      0.84      0.85       866
 weighted avg       0.86      0.85      0.84       866


Naive Bayes with TF-IDF Accuracy: 0.8302540415704388
               precision    recall  f1-score   support

entertainment       0.92      0.90      0.91       237
       nation       0.80      0.85      0.82       357
       others       0.80      0.74      0.77       272

     accuracy                           0.83       866
    macro avg       0.84      0.83      0.83       866
 weighted avg       0.83      0.83      0.83       866


Naive Bayes with Word2Vec Accuracy: 0.6570438799076213
               precision    recall  f1-score   support

  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest Model Training and Evaluation

In [ ]:
# Random Forest with FastText
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(f'\nRandom Forest with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Random Forest with TF-IDF
rf_classifier.fit(TX_train, Ty_train)
Ty_pred = rf_classifier.predict(TX_test)
print(f'\nRandom Forest with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Random Forest with Word2Vec
rf_classifier.fit(WX_train, Wy_train)
Wy_pred = rf_classifier.predict(WX_test)
print(f'\nRandom Forest with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Random Forest with FastText Accuracy: 0.9110854503464203
               precision    recall  f1-score   support

entertainment       0.98      0.94      0.96       237
       nation       0.91      0.89      0.90       357
       others       0.86      0.91      0.88       272

     accuracy                           0.91       866
    macro avg       0.92      0.91      0.91       866
 weighted avg       0.91      0.91      0.91       866


Random Forest with TF-IDF Accuracy: 0.8175519630484989
               precision    recall  f1-score   support

entertainment       0.89      0.86      0.88       237
       nation       0.78      0.87      0.82       357
       others       0.81      0.70      0.75       272

     accuracy                           0.82       866
    macro avg       0.83      0.81      0.82       866
 weighted avg       0.82      0.82      0.82       866


Random Forest with Word2Vec Accuracy: 0.7979214780600462
               precision    recall  f1-score   suppo